<a href="https://colab.research.google.com/github/reidnersousa/IFB_Arquitetura-de-Computadores-II_2023-1/blob/main/C%C3%B3pia_de_Introducao_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introdução a Linguagem de Programação CUDA**
Professor João Victor de A. Oliveira
Instituto Federal de Brasília

Antes de tudo, vamos ativar nossa GPU. No menu superior, vá em *Runtime --> Change runtime type* e selecione o acelerador de hardware "GPU".

Iremos começar com o algoritmo máis básico de todos: um **Hello World**


In [ ]:
%%writefile helloWorld.cu

#include<stdio.h>

int main (){
  printf("hello World");

  return 0;
}

Writing helloWorld.cu


Para compilarmos um código em c/C++ CUDA usaremos o compilador **nvcc**

In [ ]:
!nvidia-smi
!nvcc --version

Mon Jul 10 20:27:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvcc helloWorld.cu -o helloWorld

In [ ]:
!./helloWorld

hello World

Dois novos elementos sintáticos são necessários para adicionarmos uma função a ser usada na placa gráfica:

## \_\_global\_\_

Indica que a função irá rodar na placa gráfica (*Device*)

## mykernel<<<1,1>>>

indica que a função *mykernel* irá ser executada no *device*

Por enquanto iremos




In [ ]:
%%writefile helloDeviceOnly.cu

#include<stdio.h>

__global__ void mykernel(void) {
}


int main(void) {

mykernel<<<1,1>>>();
printf("Hello World!\n");
return 0;

}


Writing helloDeviceOnly.cu


In [ ]:
!nvcc helloDeviceOnly.cu -o helloDeviceOnly
!./helloDeviceOnly

Hello World!


In [ ]:
%%writefile soma.cu

#include<stdio.h>

__global__ void add(int *a, int *b, int *c) {
    *c = *a + *b;
}



int main(void) {
  int a, b, c;              // host copies of a, b, c
  int *d_a, *d_b, *d_c;     // device copies of a, b, c
  int size = sizeof(int);


  // Allocate space for device copies of a, b, c
  cudaMalloc((void **)&d_a, size);
  cudaMalloc((void **)&d_b, size);
  cudaMalloc((void **)&d_c, size);

  a = 2;
  b = 7;

  // Copy inputs to device
  cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
  cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);

  // Launch add() kernel on GPU
  add<<<1,1>>>(d_a, d_b, d_c);

  // Copy result back to host
  cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);

  printf("%d\n",c);

  // Cleanup
  cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);




return 0;

}

Writing soma.cu


In [ ]:
!nvcc soma.cu -o soma
!./soma

9


A grande vantagem da execução de programas na GPU é sua alta capacidade de paralelismo. No exemplo a seguir iremos rodar a função add **N** vezes.

Para isso iremos alterar a chamada da função add de:


## add <<< 1, 1 >>> (  );

para

## add <<< ***N***, 1 >>> (   );

Com isso agora podemos realizar uma função de soma em todos os elementos de um vetor.

Já no corpo da função, iremos alterar a soma para a seguinte instrução:


## c[blockIdx.x] = a[blockIdx.x] + b[blockIdx.x];

Cada invocação de add se refere a um **Bloco** referenciado por seu índice de bloco chamado **blockIdx.x**.

Um conjunto de blocos é chamado de **grid**.


In [ ]:
%%writefile somaVetorial.cu

#include<stdio.h>
#include<time.h>
#include<stdlib.h>



__global__ void add(int *a, int *b, int *c) {
	c[blockIdx.x] = a[blockIdx.x] + b[blockIdx.x];
}


void random_ints(int *a, int N){
	int i;

	for (i=0;i < N; i++){
		a[i] = rand() % 100;
    printf("%3d ",a[i]);
	}
  puts("");
}



#define N 10

int main(void) {
	int *a, *b, *c; // host copies of a, b, c
	int *d_a, *d_b, *d_c; // device copies of a, b, c
	int size = N * sizeof(int);
	int i;


	// Alloc space for device copies of a, b, c
	cudaMalloc((void **)&d_a, size);
	cudaMalloc((void **)&d_b, size);
	cudaMalloc((void **)&d_c, size);

	srand(time(NULL));

	// Alloc space for host copies of a, b, c and setup input values
	a = (int *)malloc(size); random_ints(a, N);
	b = (int *)malloc(size); random_ints(b, N);
	c = (int *)malloc(size);



	// Copy inputs to device
	cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
	cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

	// Launch add() kernel on GPU with N blocks
	add<<<N,1>>>(d_a, d_b, d_c);

	// Copy result back to host
	cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);


	for (i = 0; i< N-1; i++){
		printf("%d,",c[i]);
	}
	printf("%d\n",c[i]);


	// Cleanup
	free(a); free(b); free(c);
	cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);


	return 0;
	}

Overwriting somaVetorial.cu


In [ ]:
!nvcc somaVetorial.cu -o somaVetorial
!./somaVetorial

 41  39  85  74  96  57   4  66  81  23 
 30  34  59  74   6  49  69  94   8  38 
71,73,144,148,102,106,73,160,89,61


In [ ]:
%%writefile somaVetorial_threads.cu

#include<stdio.h>
#include<time.h>
#include<stdlib.h>



__global__ void add(int *a, int *b, int *c) {
	c[threadIdx.x] = a[threadIdx.x] + b[threadIdx.x];
}


void random_ints(int *a, int N){
	int i;

	for (i=0;i < N; i++){
		a[i] = rand() % 10;
    printf("%3d ",a[i]);
	}
  puts("");
}



#define N 10

int main(void) {
	int *a, *b, *c; // host copies of a, b, c
	int *d_a, *d_b, *d_c; // device copies of a, b, c
	int size = N * sizeof(int);
	int i;


	// Alloc space for device copies of a, b, c
	cudaMalloc((void **)&d_a, size);
	cudaMalloc((void **)&d_b, size);
	cudaMalloc((void **)&d_c, size);

	srand(time(NULL));

	// Alloc space for host copies of a, b, c and setup input values
	a = (int *)malloc(size); random_ints(a, N);
	b = (int *)malloc(size); random_ints(b, N);
	c = (int *)malloc(size);



	// Copy inputs to device
	cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
	cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

	// Launch add() kernel on GPU with N blocks
	add<<<1,N>>>(d_a, d_b, d_c);

	// Copy result back to host
	cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);


	for (i = 0; i< N-1; i++){
		printf("%d,",c[i]);
	}
	printf("%d\n",c[i]);


	// Cleanup
	free(a); free(b); free(c);
	cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);


	return 0;
	}

Overwriting somaVetorial_threads.cu


In [ ]:
!nvcc somaVetorial_threads.cu -o somaVetorial_threads
!./somaVetorial_threads

  3   5   3   9   7   4   7   6   7   5 
  0   0   3   9   1   9   9   9   2   5 
3,5,6,18,8,13,16,15,9,10


Podemos combinar o threads e blocos.

Para cada bloco teremos uma quantidade de threads. Para acessarmos cada elemento podemos calcular um índice que leva em consideração a quantidade de **M** threads por bloco.

int index =  threadIdx.x + blockIdx.x * M

Identificaremos a quantidade de threads por bloco usando a variável **blockDim.x**

int index =  threadIdx.x + blockIdx.x * blockDim.x



In [ ]:
%%writefile SomaVetorial_blocks_threads.cu

#include<stdio.h>
#include<time.h>
#include<stdlib.h>



__global__ void add(int *a, int *b, int *c) {

  int index = threadIdx.x + blockIdx.x * blockDim.x;
	c[index] = a[index] + b[index];

}


void random_ints(int *a, int N){
	int i;

	for (i=0;i < N; i++){
		a[i] = rand() % 10;
    printf("%-3d ",a[i]);
	}
  puts("");
}



#define N (128)
#define THREADS_PER_BLOCK 8

int main(void) {
	int *a, *b, *c; // host copies of a, b, c
	int *d_a, *d_b, *d_c; // device copies of a, b, c
	int size = N * sizeof(int);
	int i;


	// Alloc space for device copies of a, b, c
	cudaMalloc((void **)&d_a, size);
	cudaMalloc((void **)&d_b, size);
	cudaMalloc((void **)&d_c, size);

	srand(time(NULL));

	// Alloc space for host copies of a, b, c and setup input values
	a = (int *)malloc(size); random_ints(a, N);
	b = (int *)malloc(size); random_ints(b, N);
	c = (int *)malloc(size);



	// Copy inputs to device
	cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
	cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

	// Launch add() kernel on GPU with N blocks
	add<<<N/THREADS_PER_BLOCK, THREADS_PER_BLOCK>>>(d_a, d_b, d_c);

	// Copy result back to host
	cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);


	for (i = 0; i< N-1; i++){
		printf("%-3d,",c[i]);
	}
	printf("%-3d\n",c[i]);


	// Cleanup
	free(a); free(b); free(c);
	cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);


	return 0;
	}


Overwriting SomaVetorial_blocks_threads.cu


In [ ]:
!nvcc SomaVetorial_blocks_threads.cu -o SomaVetorial_blocks_threads
!./SomaVetorial_blocks_threads

4   1   5   1   0   9   1   2   3   4   8   9   2   2   4   6   5   5   3   2   7   1   3   2   8   5   7   0   1   6   1   7   9   6   8   9   7   9   3   1   5   1   2   7   5   8   3   0   3   6   5   3   7   8   7   8   6   5   0   7   1   1   5   2   9   3   1   7   5   4   0   0   7   4   8   3   3   3   3   6   2   0   1   1   9   9   9   5   4   1   4   7   2   9   9   4   5   2   3   0   8   5   2   5   9   2   0   4   6   6   3   8   6   4   9   7   3   1   4   9   2   9   8   7   8   9   3   3   
1   8   5   1   3   0   9   4   2   9   9   0   5   4   0   4   8   2   1   4   3   8   5   7   7   6   6   7   5   9   3   9   9   0   0   2   0   1   9   5   1   0   5   8   4   6   4   4   8   6   0   3   4   6   2   3   2   9   2   9   0   5   8   0   8   1   4   8   2   3   3   5   3   1   6   9   9   0   4   9   8   4   2   4   2   6   9   6   5   2   6   8   9   6   0   7   9   4   6   2   0   1   9   5   2   7   5   3   8   1   2   8   7   6   3   2   3   2   8   0   4   6  

Nem sempre **(na maioria das vezes)** não teremos N sendo um múltiplo de **blockDim.x**.

Neste caso teremos que adaptar nosso código na chamada da função e dentro da função:

In [ ]:
%%writefile SomaVetorial_blocks_threads_N_variavel.cu

#include<stdio.h>
#include<time.h>
#include<stdlib.h>
#include<cuda_runtime.h>



__global__ void add(int *a, int *b, int *c, int n) {

  int index = threadIdx.x + blockIdx.x * blockDim.x;
	if (index < n){
    c[index] = a[index] + b[index];
  }

}


void random_ints(int *a, int N){
	int i;

	for (i=0;i < N; i++){
		a[i] = rand() % 10;
    printf("%-3d ",a[i]);
	}
  puts("");
}



#define N (130)
#define THREADS_PER_BLOCK 8

int main(void) {
	int *a, *b, *c; // host copies of a, b, c
	int *d_a, *d_b, *d_c; // device copies of a, b, c
	int size = N * sizeof(int);
	int i;


	// Alloc space for device copies of a, b, c
	cudaMalloc((void **)&d_a, size);
	cudaMalloc((void **)&d_b, size);
	cudaMalloc((void **)&d_c, size);

	srand(time(NULL));

	// Alloc space for host copies of a, b, c and setup input values
	a = (int *)malloc(size); random_ints(a, N);
	b = (int *)malloc(size); random_ints(b, N);
	c = (int *)malloc(size);



	// Copy inputs to device
	cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
	cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

	// Launch add() kernel on GPU with N blocks
	add<<< ((N + THREADS_PER_BLOCK-1) / THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_a, d_b, d_c,N);

  // Sincronização do dispositivo
  cudaDeviceSynchronize();

	// Copy result back to host
	cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);



	for (i = 0; i < N-1; i++){
		printf("%-3d,",c[i]);
	}
	printf("%-3d\n",c[i]);

	// Cleanup
	free(a); free(b); free(c);
	cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);


	return 0;
	}


Writing SomaVetorial_blocks_threads_N_variavel.cu


In [ ]:
!nvcc SomaVetorial_blocks_threads_N_variavel.cu -o SomaVetorial_blocks_threads_N_variavel
!./SomaVetorial_blocks_threads_N_variavel

4   7   9   6   4   6   8   7   4   4   9   6   6   2   3   6   5   3   8   4   2   2   6   0   8   0   3   7   6   6   0   2   3   1   1   9   0   9   9   4   5   8   0   1   0   5   7   7   8   7   4   2   0   2   3   0   2   8   7   0   4   9   3   0   0   6   9   0   5   8   6   2   8   8   3   1   3   2   0   4   1   4   6   3   6   1   3   8   1   0   1   8   9   4   8   2   0   9   4   7   0   1   9   8   9   4   9   5   6   0   9   7   6   7   3   3   1   6   3   2   9   4   0   0   0   0   2   0   0   9   
9   2   0   0   0   1   4   2   6   2   4   7   2   0   5   5   3   6   1   9   0   0   3   3   1   6   3   5   8   5   4   8   9   6   0   0   8   5   4   6   7   8   4   9   8   1   4   2   9   8   3   1   8   6   4   1   4   0   7   3   7   1   1   7   8   1   9   8   8   3   6   8   1   2   9   9   5   6   3   4   4   6   6   2   5   2   4   9   2   3   2   0   4   5   9   4   9   8   4   7   1   3   5   4   5   7   5   1   3   1   7   7   9   3   1   4   6   7   6   0  

ta certo


In [ ]:
%%writefile stencil1d.cu
#include<stdio.h>
#include<time.h>
#include<stdlib.h>

int const N =200;
int const THREADS_PER_BLOCK =8;
int const RADIUS =3;

__global__ void stencil_1d(int *in, int *out){

	__shared__ int temp[N + 2 * RADIUS];
	int gindex = threadIdx.x + blockIdx.x * blockDim.x;
	int lindex = threadIdx.x + 3;

	temp[lindex] = in[gindex];
	if (threadIdx.x < 3){
		if (gindex-RADIUS >= 0){
			temp[lindex - RADIUS] = in[gindex - RADIUS];
		}
		else{
			temp[lindex - RADIUS] = 0;
		}
		if (gindex + blockDim.x > N){
			temp[lindex + RADIUS] = 0;
		}
		else{
			temp[lindex + RADIUS] = in[gindex + blockDim.x];
		}

	}

	__syncthreads();


	// if (threadIdx.x == 5){
	// 	printf("%d\n%d --\n",lindex, temp[lindex]);
	// 	for (int i=0; i< 8+2*3; i++){
	// 		printf("%-3d",temp[i]);
	// 	}
	// 	printf("\n\n\n");
	// }


	int result = 0;
	for (int offset = -RADIUS; offset <= RADIUS; offset++){
		result += temp[lindex + offset];
	}

	out[gindex] = result;

}


void random_ints(int *a, int N){
	int i;

	for (i=0;i < N; i++){
		a[i] = rand() % 10;
    printf("%-3d ",a[i]);
	}
  puts("");
}




int main(void) {
	int *in, *out;
	int *d_in, *d_out;
	int size = N * sizeof(int);
	int i;


	cudaMalloc((void **)&d_in, size);
	cudaMalloc((void **)&d_out, size);

	srand(1);


	in = (int *)malloc(size); random_ints(in, N);
	out = (int *)malloc(size);


	// Copy inputs to device
	cudaMemcpy(d_in, in, size, cudaMemcpyHostToDevice);
	cudaMemcpy(d_out, out, size, cudaMemcpyHostToDevice);

	stencil_1d<<<((N + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK), THREADS_PER_BLOCK>>>(d_in, d_out);

	// Copy result back to host
	cudaMemcpy(out, d_out, size, cudaMemcpyDeviceToHost);

	cudaDeviceSynchronize();


	for (i = 0; i< N-1; i++){
		printf("%-3d,",out[i]);
	}
	printf("%-3d\n",out[i]);


	// Cleanup
	free(in); free(out);
	cudaFree(d_in); cudaFree(d_out);


	return 0;
}

Writing stencil1d.cu


In [ ]:
!nvcc stencil1d.cu -o stencil1d
!./stencil1d

3   6   7   5   3   5   6   2   9   1   2   7   0   9   3   6   0   6   2   6   1   8   7   9   2   0   2   3   7   5   9   2   2   8   9   7   3   6   1   2   9   3   1   9   4   7   8   4   5   0   3   6   1   0   6   3   2   0   6   1   5   5   4   7   6   5   6   9   3   7   4   5   2   5   4   7   4   4   3   0   7   8   6   8   8   4   3   1   4   9   2   0   6   8   9   2   6   6   4   9   5   0   4   8   7   1   7   2   7   2   2   6   1   0   6   1   5   9   4   9   0   9   1   7   7   1   1   5   9   7   7   6   7   3   6   5   6   3   9   4   8   1   2   9   3   9   0   8   8   5   0   9   6   3   8   5   6   1   1   5   9   8   4   8   1   0   3   0   4   4   4   4   7   6   3   1   7   5   9   6   2   1   7   8   5   7   4   1   8   5   9   7   5   3   8   8   3   1   8   9   
21 ,24 ,29 ,35 ,34 ,28 ,21 ,16 ,32 ,27 ,30 ,31 ,28 ,27 ,25 ,18 ,32 ,24 ,29 ,30 ,39 ,33 ,31 ,25 ,31 ,30 ,28 ,28 ,28 ,28 ,26 ,23 ,42 ,40 ,37 ,36 ,36 ,28 ,19 ,12 ,31 ,29 ,35 ,41 ,36 ,33 ,32 ,23 ,33 ,27 